In [520]:
from astropy.io import ascii
import numpy as np
import gurobipy as gp
from gurobipy import *
import random

random.seed(10)

Placa_madre = ascii.read("Opti - Placa Madre (1).dat")
Memorias = ascii.read("Opti - RAM (2).dat")
Almacenamiento = ascii.read("Opti - Almacenamiento SSD (3).dat")
Procesador = ascii.read("Opti - Procesador (4).dat")
Fuente_de_poder = ascii.read("Opti - Fuente de poder (5).dat")
Tarjeta_de_video = ascii.read("Opti - Tarjeta de video (6).dat")
Gabinete = ascii.read("Opti - Gabinete (7).dat")

Componentes = [Placa_madre, Memorias, Almacenamiento, Procesador, Fuente_de_poder, Tarjeta_de_video, Gabinete]
comps = ["Placa madre", "Memoria", "SSD", "CPU", "Fuente de poder", "Tarjeta de video", "Gabinete"]
tiendas = ['Las Condes','Manuel Montt','Mall Arauco Maipu','Mall Plaza Alameda','Chillan']

I_ = range(0, 7)
M_ = [range(0, len(Componentes[0])),range(0, len(Componentes[1])),range(0, len(Componentes[2])),
     range(0, len(Componentes[3])),range(0, len(Componentes[4])),range(0, len(Componentes[5])), range(0, len(Componentes[6]))]
T_ = range(0, 5)

c = {(i,m,t) : Componentes[i]['Precio ($)'][m] for i in I_ for m in M_[i] for t in T_}
k = {t : 1 for t in T_}
p = {(i,m): Componentes[i]['Potencia (W)'][m] for i in [1,3,4,5] for m in M_[i]}
e = {(i,m): Componentes[i]['Especificacion'][m] for i in [1,2,3,5] for m in M_[i]}
E = {1: 8, 2: 500, 3: 4*3.7, 5: 1}
r = {m: Componentes[0]['Slots de memoria'][m] for m in M_[0]}
a = {m: Componentes[0]['Almacenamiento'][m] for m in M_[0]}
S = {(i,m,t): Componentes[i][tiendas[t]][m] for i in I_ for m in M_[i] for t in T_}
f = {(i,m): Componentes[i]['Factor de forma'][m] for i in [0,4,6] for m in M_[i]}
tam = {(i,m): Componentes[i]['Tamano de tarjeta'][m] for i in [5,6] for m in M_[i]}
sl = {(i,m): Componentes[i]['Uso de slots'][m] for i in [5,6] for m in M_[i]}

R = {(1,m): [i for i in range(len(Componentes[1]['Precio ($)'])) if 
             Componentes[1]['Tipo de memoria'][i] != Componentes[0]['Tipo de memoria'][m] or
             Componentes[1]['Frecuencia (MHz)'][i] < Componentes[0]['Velocidad minima (MHz)'][m] or
             Componentes[1]['Frecuencia (MHz)'][i] > Componentes[0]['Velocidad maxima (MHz)'][m]   
            ]
             for m in M_[0]}

for m in M_[0]:
    R[3,m] = [i for i in range(len(Componentes[3]['Precio ($)'])) if 
             Componentes[3]['Marca'][i] != Componentes[0]['Plataforma'][m]]
    
H = {(1,m): [i for i in range(len(Componentes[1]['Precio ($)'])) if 
             Componentes[1]['Frecuencia (MHz)'][i] > Componentes[3]['Velocidad maxima (MHz)'][m] or
             Componentes[1]['Tipo de memoria'][i] != Componentes[3]['Tipos de memoria'][m] + ' DIMM']
             for m in M_[3]}

phi = {t: 1 for t in T_}


l = []
for i in I_:
    for m in M_[i]:
        for t in T_:
            l.append((i,m,t))
            

In [521]:
# MODELO
model = Model("Costo_Mínimo_Computador")
            
# -------------------------------------------
# DEFINICIÓN DE VARIABLES
x = model.addVars(l,vtype = GRB.INTEGER, name = "Componente", lb = 0.0)
y = model.addVars(T_,vtype = GRB.BINARY, name = "envio")
z = model.addVars(T_,vtype = GRB.BINARY, name = "envio_gratis")

# -------------------------------------------
# RESTRICCIONES

# Se debe comprar solo una unidad de algunos componentes
model.addConstrs((quicksum(quicksum(x[i,m,t] for m in M_[i]) for t in T_) == 1 for i in [0,3,4,6]), name = "Componentes_unitarios")

# Se debe comprar a lo mas una tarjeta de video
model.addConstrs((quicksum(quicksum(x[i,m,t] for m in M_[i]) for t in T_) <= 1 for i in [5]), name = "Componentes_unitarios")

# Respetar el limite de espacios de RAM en la placa madre
model.addConstr((quicksum(quicksum(x[1,m,t] for m in M_[1]) for t in T_) <= quicksum(quicksum(r[m] * x[0,m,t] for m in M_[0]) for t in T_)), name = "Respetar_RAM")

# Respetar el limite de espacios de almacenamiento en la placa madre
model.addConstr((quicksum(quicksum(x[2,m,t] for m in M_[2]) for t in T_) <= quicksum(quicksum(a[m] * x[0,m,t] for m in M_[0]) for t in T_)), name = "Respetar_almacenamiento")

# Especificaciones
model.addConstrs((quicksum(quicksum(e[i,m] * x[i,m,t] for m in M_[i]) for t in T_) >= E[i] for i in [1,2,3,5]), name = "Especificaciones_minimas")

# Respetar el limite de potencia
model.addConstr((quicksum(quicksum(quicksum(p[i,m] * x[i,m,t] for m in M_[i]) for t in T_) for i in [1,3,4,5]) >= 0), name = "Respetar_limite_potencia")

# Cobrar envio
A = 1e10
model.addConstrs((quicksum(quicksum(x[i,m,t] for m in M_[i]) for i in I_) <= A * y[t] for t in T_), name = "cobrar_envio")

# envio grtis
model.addConstrs((quicksum(quicksum(c[i,m,t] * x[i,m,t] for m in M_[i]) for i in I_) >= phi[t] * z[t] for t in T_), name = "Envio_gratis")

# compatibilidad placa madre con otros componentes
model.addConstrs((A*(1 - quicksum(x[0,m,t] for t in T_)) >= quicksum(x[i,k,t] for t in T_) for m in M_[0] for i in [1,3] for k in R[i,m]), name = "Compatibilidad_placa_madre")

# compatibilidad procesador con otros componentes
model.addConstrs((A*(1 - quicksum(x[3,m,t] for t in T_)) >= quicksum(x[i,k,t] for t in T_) for m in M_[3] for i in [1] for k in H[i,m]), name = "Compatibilidad_procesador")

# Respetar disponibilidad en tienda
model.addConstrs((x[i,m,t] <= S[i,m,t] for i in I_ for m in M_[i] for t in T_), name = 'Stock')

# Respetar factor de forma del gabinete
model.addConstrs((quicksum(quicksum(f[6,m]*x[6,m,t] for m in M_[6]) for t in T_) >= quicksum(quicksum(f[i,m]*x[i,m,t] for m in M_[i]) for t in T_) for i in [0,4]), name = 'Factor_de_forma')

# Respetar largo de tarjeta de video
model.addConstr((quicksum(quicksum(tam[6,m]*x[6,m,t] for m in M_[6]) for t in T_) >= quicksum(quicksum(tam[5,m]*x[5,m,t] for m in M_[5]) for t in T_)), name = 'Respetar_largo_tarjeta_de_video')

# Respetar ancho de tarjeta de video
model.addConstr((quicksum(quicksum(sl[6,m]*x[6,m,t] for m in M_[6]) for t in T_) >= quicksum(quicksum(sl[5,m]*x[5,m,t] for m in M_[5]) for t in T_)), name = 'Respetar_ancho_tarjeta_de_video')


# Naturaleza variables
model.addConstrs((x[i,m,t] >= 0 for i in I_ for m in M_[i] for t in T_),name = "Naturaleza_x_it")
model.addConstrs((y[t] >= 0 for t in T_),name = "Naturaleza_y_it")                 
model.addConstrs((z[t] >= 0 for t in T_),name = "Naturaleza_z_ijt")  
# como ya fue definido antes que z era variable binaria, es necesario poner su restriccion de naturaleza?
#model.addConstrs((z[i,t]=?),name = "Naturaleza z_it")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [522]:
obj = quicksum((y[t] - z[t])*k[t]  + quicksum(quicksum(c[i,m,t] * x[i,m,t] for m in M_[i]) for i in I_) for t in T_)
                 
model.setObjective(obj, GRB.MINIMIZE)
                 

# Actualizar modelo
model.update()
       
# Escribir formulación
model.write("Modelo {}.lp".format(model.ModelName)) 

# Optimizar
model.optimize()



#################################

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 2259 rows, 760 columns and 13050 nonzeros
Model fingerprint: 0x93204ef9
Variable types: 0 continuous, 760 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [1e+00, 5e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2086 rows and 576 columns
Presolve time: 0.05s
Presolved: 173 rows, 184 columns, 1475 nonzeros
Variable types: 0 continuous, 184 integer (150 binary)
Found heuristic solution: objective 520480.00000

Root relaxation: objective 4.310428e+05, 26 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 431042.842    0   14 520480.000 431042.842  17.2%     -    0s
H    0     0    

In [523]:
# Mostrar resultados
print(f"\n### Ganancia óptima: {model.objVal} calculado en {model.runtime} segundos. ###\n")


### Ganancia óptima: 440330.0 calculado en 0.18989181518554688 segundos. ###



In [524]:
model.printAttr("x") # all non-zero solution values


    Variable            x 
-------------------------
Componente[0,1,1]            1 
Componente[1,10,0]            1 
Componente[2,14,0]            1 
Componente[3,10,1]            1 
Componente[4,0,1]            1 
Componente[5,0,4]            1 
Componente[6,1,3]            1 
    envio[0]            1 
    envio[1]            1 
    envio[3]            1 
    envio[4]            1 
envio_gratis[0]            1 
envio_gratis[1]            1 
envio_gratis[3]            1 
envio_gratis[4]            1 


In [525]:
for i in I_:
    for m in M_[i]:
        for t in T_:
            if x[i,m,t].x>0:
                if tiendas[t] != 'Chillan':
                    print('Compra la {} ID {} en la sucursal en {}'.format(comps[i],Componentes[i]['ID'][m], tiendas[t]))
                else:
                    print('Compra la {} ID {} en la sucursal en {} y de paso compra unas longanizas'.format(comps[i],Componentes[i]['ID'][m], tiendas[t]))

print('Con un costo total de {}'.format(model.objVal))                

Compra la Placa madre ID 36926 en la sucursal en Manuel Montt
Compra la Memoria ID 37040 en la sucursal en Las Condes
Compra la SSD ID 29235 en la sucursal en Las Condes
Compra la CPU ID 32738 en la sucursal en Manuel Montt
Compra la Fuente de poder ID 25210 en la sucursal en Manuel Montt
Compra la Tarjeta de video ID 35236 en la sucursal en Chillan y de paso compra unas longanizas
Compra la Gabinete ID 36559 en la sucursal en Mall Plaza Alameda
Con un costo total de 440330.0


In [406]:
r y de paso compra unas longanizas

SyntaxError: invalid syntax (<ipython-input-406-29eb34a1e62f>, line 1)

In [463]:
Componentes[5]['Tamano de ']

ID,MARCA,GPU,Las Condes,Manuel Montt,Mall Arauco Maipu,Mall Plaza Alameda,Chillan,Precio ($),TIPO DE MEMORIA,Memoria,Potencia (W),Especificacion,Tamano de tajeta,Uso de slots
int32,str8,str6,int32,int32,int32,int32,int32,int32,str5,int32,int32,int32,int32,int32
35236,Galax,NVIDIA,0,0,0,0,1,49990,DDR3,1,-300,1,1,1
38685,Gigabyte,NVIDIA,5,0,0,3,2,52190,GDDR5,2,-300,2,1,1
28507,MSI,NVIDIA,1,0,0,0,0,54990,DDR3,1,-19,1,1,1
36075,Asus,NVIDIA,7,8,6,6,3,59990,GDDR5,1,-19,1,1,1
35258,Asus,NVIDIA,1,0,0,0,1,64990,GDDR5,2,-19,2,1,1
37862,PNY,NVIDIA,1,0,0,0,1,139990,GDDR5,2,-30,2,1,1
36974,AMD,AMD,0,0,0,0,1,163390,GDDR5,2,-35,2,2,2
29276,Gigabyte,NVIDIA,3,2,0,3,0,169990,GDDR5,4,-300,4,2,2
38684,Gigabyte,NVIDIA,5,6,1,0,2,171690,GDDR6,4,-300,4,1,1
